In [1]:
%cd ../..

/home/ivanstefanov/Repositories/used-cars-price-prediction


In [2]:
import pandas as pd

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [4]:
import tensorflow.keras as tf
import tensorflow.keras.layers

2023-02-06 22:18:38.389572: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-06 22:18:38.696795: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-02-06 22:18:38.696844: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-02-06 22:18:40.568041: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [21]:
from price_predictor.data_preparation.feature_engineering import add_class_feature
from price_predictor.data_preparation.pick_or_drop_features import (
    pick_by_datatypes,
    drop_anonymous_features,
    drop_categorical_features,
    drop_by_name
)
from price_predictor.evaluate import evaluate_model, evaluate

In [15]:
data_all = pd.read_csv('data/cars.csv')
data_all.dropna(inplace=True)

## Try numerical features only

In [7]:
data = pick_by_datatypes(data_all, ['numerical', 'boolean'])
data

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,190000,2010,False,2.5,False,10900.00,False,9,13,False,True,True,True,False,True,False,True,True,True,16
1,290000,2002,False,3.0,False,5000.00,True,12,54,False,True,False,False,True,True,False,False,False,True,83
2,402000,2001,False,2.5,False,2800.00,True,4,72,False,True,False,False,False,False,False,False,True,True,151
3,10000,1999,False,3.0,False,9999.00,True,9,42,True,False,False,False,False,False,False,False,False,False,86
4,280000,2001,False,2.5,False,2134.11,True,14,7,False,True,False,True,True,False,False,False,False,True,7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,290000,2000,False,3.5,False,2750.00,True,5,85,False,True,False,False,True,True,False,False,True,True,301
38527,321000,2004,False,2.2,False,4800.00,True,4,20,False,True,False,False,True,True,False,False,True,True,317
38528,777957,2000,False,3.5,False,4300.00,False,3,63,False,True,False,False,True,True,False,False,True,True,369
38529,20000,2001,False,2.0,False,4000.00,True,7,156,False,True,False,False,False,False,False,False,False,True,490


In [8]:
scaler = MinMaxScaler()
scaled_data = pd.DataFrame(scaler.fit_transform(data), index=data.index, columns=data.columns)
scaled_data

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,feature_0,feature_1,feature_2,feature_3,feature_4,feature_5,feature_6,feature_7,feature_8,feature_9,duration_listed
0,0.190000,0.883117,0.0,0.294872,0.0,0.217984,0.0,0.094118,0.006452,0.0,1.0,1.0,1.0,0.0,1.0,0.0,1.0,1.0,1.0,0.007168
1,0.290000,0.779221,0.0,0.358974,0.0,0.099982,1.0,0.129412,0.028495,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.037186
2,0.402000,0.766234,0.0,0.294872,0.0,0.055981,1.0,0.035294,0.038172,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.067652
3,0.010000,0.740260,0.0,0.358974,0.0,0.199964,1.0,0.094118,0.022043,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.038530
4,0.280000,0.766234,0.0,0.294872,0.0,0.042663,1.0,0.152941,0.003226,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.003136
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,0.290000,0.753247,0.0,0.423077,0.0,0.054981,1.0,0.047059,0.045161,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.134857
38527,0.321000,0.805195,0.0,0.256410,0.0,0.095982,1.0,0.035294,0.010215,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.142025
38528,0.777957,0.753247,0.0,0.423077,0.0,0.085982,0.0,0.023529,0.033333,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.165323
38529,0.020000,0.766234,0.0,0.230769,0.0,0.079982,1.0,0.070588,0.083333,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.219534


In [10]:
X_train, X_test, y_train, y_test = train_test_split(scaled_data.drop('price_usd', axis=1), scaled_data['price_usd'])

## Two hidden layer dense model with 256 nodes each

In [12]:
model = tf.Sequential()
model.add(tf.layers.Dense(len(X_train)))
model.add(tf.layers.Dense(256, activation='relu'))
model.add(tf.layers.Dense(256, activation='relu'))
model.add(tf.layers.Dense(1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

Epoch 1/20
226/226 [==============================] - 49s 209ms/step - loss: 0.0649
Epoch 2/20
226/226 [==============================] - 47s 206ms/step - loss: 0.0450
Epoch 3/20
226/226 [==============================] - 52s 230ms/step - loss: 0.0421
Epoch 4/20
226/226 [==============================] - 52s 228ms/step - loss: 0.0407
Epoch 5/20
226/226 [==============================] - 47s 207ms/step - loss: 0.0400
Epoch 6/20
226/226 [==============================] - 46s 202ms/step - loss: 0.0377
Epoch 7/20
226/226 [==============================] - 46s 205ms/step - loss: 0.0375
Epoch 8/20
226/226 [==============================] - 45s 200ms/step - loss: 0.0374
Epoch 9/20
226/226 [==============================] - 66s 294ms/step - loss: 0.0369
Epoch 10/20
226/226 [==============================] - 54s 240ms/step - loss: 0.0383
Epoch 11/20
226/226 [==============================] - 47s 210ms/step - loss: 0.0375
Epoch 12/20
226/226 [==============================] - 47s 206ms/step - lo

In [ ]:
model.fit(X_train, y_train, batch_size = 128, epochs=20)

In [13]:
evaluate_model(model, X_test, y_test)

301/301 [==============================] - 6s 20ms/step


{'rmse': 0.0626239354396075,
 'mae': 0.03476074184260313,
 'r2': 0.7608392242752923}

In [14]:
# check for overfit
evaluate_model(model, X_train, y_train)

903/903 [==============================] - 19s 21ms/step


{'rmse': 0.06164807227474057,
 'mae': 0.03469036867756325,
 'r2': 0.7703911069553833}

## Its not overfitted, but not great results

## Lets try the brute force approach

In [22]:
data = drop_by_name(data_all, 'model_name')

In [24]:
dummies = pd.get_dummies(data)
scaled_dummies = pd.DataFrame(MinMaxScaler().fit_transform(dummies), index=dummies.index, columns=dummies.columns)
scaled_dummies

,odometer_value,year_produced,engine_has_gas,engine_capacity,has_warranty,price_usd,is_exchangeable,number_of_photos,up_counter,feature_0,...,state_owned,drivetrain_all,drivetrain_front,drivetrain_rear,location_region_Брестская обл.,location_region_Витебская обл.,location_region_Гомельская обл.,location_region_Гродненская обл.,location_region_Минская обл.,location_region_Могилевская обл.
0,0.190000,0.883117,0.0,0.294872,0.0,0.217984,0.0,0.094118,0.006452,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.290000,0.779221,0.0,0.358974,0.0,0.099982,1.0,0.129412,0.028495,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.402000,0.766234,0.0,0.294872,0.0,0.055981,1.0,0.035294,0.038172,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.010000,0.740260,0.0,0.358974,0.0,0.199964,1.0,0.094118,0.022043,1.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.280000,0.766234,0.0,0.294872,0.0,0.042663,1.0,0.152941,0.003226,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38526,0.290000,0.753247,0.0,0.423077,0.0,0.054981,1.0,0.047059,0.045161,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
38527,0.321000,0.805195,0.0,0.256410,0.0,0.095982,1.0,0.035294,0.010215,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
38528,0.777957,0.753247,0.0,0.423077,0.0,0.085982,0.0,0.023529,0.033333,0.0,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
38529,0.020000,0.766234,0.0,0.230769,0.0,0.079982,1.0,0.070588,0.083333,0.0,...,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [25]:
X_train, X_test, y_train, y_test = train_test_split(scaled_dummies.drop('price_usd', axis=1), scaled_dummies['price_usd'])

In [26]:
model = tf.Sequential()
model.add(tf.layers.Dense(len(X_train)))
model.add(tf.layers.Dense(256, activation='relu'))
model.add(tf.layers.Dense(256, activation='relu'))
model.add(tf.layers.Dense(1, activation='linear'))

model.compile(loss='mean_absolute_error', optimizer='adam')

In [27]:
model.fit(X_train, y_train, batch_size = 128, epochs=20)

Epoch 1/20
226/226 [==============================] - 67s 282ms/step - loss: 0.0842
Epoch 2/20
226/226 [==============================] - 63s 281ms/step - loss: 0.0420
Epoch 3/20
226/226 [==============================] - 68s 301ms/step - loss: 0.0387
Epoch 4/20
226/226 [==============================] - 64s 282ms/step - loss: 0.0360
Epoch 5/20
226/226 [==============================] - 61s 270ms/step - loss: 0.0337
Epoch 6/20
226/226 [==============================] - 61s 272ms/step - loss: 0.0323
Epoch 7/20
226/226 [==============================] - 59s 262ms/step - loss: 0.0319
Epoch 8/20
226/226 [==============================] - 59s 263ms/step - loss: 0.0305
Epoch 9/20
226/226 [==============================] - 59s 259ms/step - loss: 0.0305
Epoch 10/20
226/226 [==============================] - 57s 251ms/step - loss: 0.0309
Epoch 11/20
226/226 [==============================] - 59s 260ms/step - loss: 0.0298
Epoch 12/20
226/226 [==============================] - 60s 267ms/step - lo

In [28]:
evaluate_model(model, X_test, y_test)

301/301 [==============================] - 7s 24ms/step


{'rmse': 0.04154221316675154,
 'mae': 0.025493666820389446,
 'r2': 0.8926651201344735}

In [29]:
# check for overfit
evaluate_model(model, X_train, y_train)

903/903 [==============================] - 21s 23ms/step


{'rmse': 0.04305783345317848,
 'mae': 0.025843051843683885,
 'r2': 0.888706406323663}

## Still not better than the Random Forest brute force though very comparable; takes eternity to train